In [1]:
import pandas as pd
import numpy as np
import kfold_and_metrics as km

from sklearn.metrics import roc_auc_score
import tensorflow as tf

tf.keras.backend.clear_session()

In [2]:
df = pd.read_csv('Lero/pre_final.csv')
tf.random.set_seed(22)
df.drop(columns=["Id"], inplace=True)
df

,patient_id,slice_thickness,pixel_spacing,subtlety,internalStructure,calcification,sphericity,margin,lobulation,spiculation,texture,diameter,surface_area,volume,malignancy
0,LIDC-IDRI-0001,2.5,0.70,5,1,6,3,3,3,4,5,32.205888,2801.428431,7869.369507,1
1,LIDC-IDRI-0001,2.5,0.70,5,1,6,4,4,5,5,5,33.188692,2198.454852,6146.438599,1
2,LIDC-IDRI-0001,2.5,0.70,5,1,6,3,2,3,3,5,31.959332,2392.704188,6594.474792,1
3,LIDC-IDRI-0001,2.5,0.70,5,1,6,5,4,1,5,4,33.669337,2573.278820,7348.411560,1
4,LIDC-IDRI-0002,1.2,0.68,2,1,6,5,1,1,1,1,30.023172,2880.758652,6576.900681,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4213,LIDC-IDRI-1011,2.5,0.85,5,1,6,3,4,2,3,5,21.909849,1121.939102,2322.662354,1
4214,LIDC-IDRI-1012,2.5,0.72,1,1,6,5,2,1,1,1,10.346859,159.437117,132.646850,-1
4215,LIDC-IDRI-1012,2.5,0.72,4,1,6,4,4,2,2,3,7.262603,151.242954,125.857838,-1
4216,LIDC-IDRI-1012,2.5,0.72,4,1,6,4,5,1,1,5,10.522030,252.306740,258.504689,-1


In [3]:
def best_nn_params(df):

    activation_possibilities = ["softmax", "relu", "sigmoid"]
    max = 0
    best = []

    for activation_hidden in activation_possibilities:
        for num_nodes_hidden_layer in range(5, len(df.columns), 10):
            for gradient in [0.001, 0.003, 0.005, 0.007, 0.01]:

                nn_model = tf.keras.models.Sequential([
                    tf.keras.layers.Input(shape=(13,)),
                    #tf.keras.layers.Flatten(input_shape=(len(df.columns),)),
                    tf.keras.layers.Dense(num_nodes_hidden_layer,activation=activation_hidden),
                    tf.keras.layers.Dense(2,activation='softmax')
                ])

                nn_model.compile(
                    optimizer=tf.keras.optimizers.SGD(gradient), 
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
                    )

                score = km.k_fold_cv_keras(compiled_model=nn_model, df=df)
                avg, std = km.weighted_avg_and_std(np.array(score["roc_auc_score"]))
                
                if avg > max:
                    best = [num_nodes_hidden_layer, activation_hidden, gradient]

    nn_model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(13,)),
        #tf.keras.layers.Flatten(input_shape=(len(df.columns))),
        tf.keras.layers.Dense(best[0],activation=best[1]),
        tf.keras.layers.Dense(2,activation='softmax')
    ])
                
    nn_model.compile(
        optimizer=tf.keras.optimizers.SGD(best[2]), 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
        )

    return nn_model


In [4]:
model = best_nn_params(df)

d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[index].append(last_fold[last_fold['patient_id']==pid])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[index].append(last_fold[last_fold['patient_id']==pid])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folds[index].append(last_fold[last_fold['patient_id']==pid])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD

...

d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD Project 1\LabIACD-Project-1\kfold_and_metrics.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df.append(folds[i])
d:\LabIACD Projeto 1\LabIACD

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10


KeyboardInterrupt: 